In [17]:
# I've created a pip package containing a wrapper for the model.
!pip install --upgrade keras-svm

Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple
Requirement already up-to-date: keras-svm in /usr/local/lib/python3.6/dist-packages (1.0.0b10)
Requirement not upgraded as not directly required: scikit-learn in /usr/local/lib/python3.6/dist-packages (from keras-svm) (0.19.1)
Requirement not upgraded as not directly required: keras in /usr/local/lib/python3.6/dist-packages (from keras-svm) (2.1.6)
Requirement not upgraded as not directly required: pyyaml in /usr/local/lib/python3.6/dist-packages (from keras->keras-svm) (3.12)
Requirement not upgraded as not directly required: h5py in /usr/local/lib/python3.6/dist-packages (from keras->keras-svm) (2.7.1)
Requirement not upgraded as not directly required: six>=1.9.0 in /usr/local/lib/python3.6/dist-packages (from keras->keras-svm) (1.11.0)
Requirement not upgraded as not directly required: numpy>=1.9.1 in /usr/local/lib/python3.6/dist-packages (from keras->keras-svm) (1.14.2)
Requirement not upgraded as not dire

In [ ]:
import keras
from keras_svm.model_svm_wrapper import ModelSVMWrapper
from keras import layers, models, backend
from keras.datasets import mnist, fashion_mnist
from keras.utils import to_categorical
from keras.models import Model
from keras.engine.topology import Layer
import matplotlib.pyplot as plt
from google.colab import files
import pickle
import json

In [ ]:
(train_images, train_labels), (test_images, test_labels) = \
    fashion_mnist.load_data() 
    # or mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

# Build a generic CNN

----
(based on https://github.com/fchollet/deep-learning-with-python-notebooks)

Creating a simple CNN with three convolutional layers.

In [ ]:
def build_model():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(10, activation='softmax'))
  model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  return model

wrapper = ModelSVMWrapper(build_model())

Let's display the architecture of our convnet so far:

In [21]:
wrapper.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_7 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 64)                36928     
__________

Prepare the data for the CNN

Train model and store intermediate test results

In [22]:
for j in range(5): 
  wrapper = ModelSVMWrapper(build_model())

  epochs = 10
  performance = {
        "with_svm@-2": [],
        "with_svm@-3": [],
        "without_svm": []
    }
  for i in range(epochs):
    print('Starting epoch: {}'.format(i + 1))
    wrapper.fit(train_images, train_labels, epochs=1, batch_size=64)
    performance["with_svm@-3"].append(wrapper.evaluate(test_images, test_labels))
    performance["without_svm"].append(
        wrapper.model.evaluate(test_images, to_categorical(test_labels))[1])

    # Try it for the different SVM
    wrapper.fit_svm(train_images, train_labels, wrapper.model.layers[-2])
    performance["with_svm@-2"].append(wrapper.evaluate(test_images, test_labels))
    
    print(performance)
  filename = 'performance{}.json'.format(j)
  print(filename)
  with  open(filename, 'w') as file:
    json.dump(performance, file)

Starting epoch: 1
Epoch 1/1
10000/10000 [==============================] - 1s 94us/step
{'with_svm@-2': [0.8876], 'with_svm@-3': [0.8999], 'without_svm': [0.8667]}
Starting epoch: 2
Epoch 1/1
10000/10000 [==============================] - 1s 91us/step
{'with_svm@-2': [0.8876, 0.8992], 'with_svm@-3': [0.8999, 0.9046], 'without_svm': [0.8667, 0.8758]}
Starting epoch: 3
Epoch 1/1
10000/10000 [==============================] - 1s 93us/step
{'with_svm@-2': [0.8876, 0.8992, 0.9077], 'with_svm@-3': [0.8999, 0.9046, 0.9097], 'without_svm': [0.8667, 0.8758, 0.8939]}
Starting epoch: 4
Epoch 1/1
10000/10000 [==============================] - 1s 94us/step
{'with_svm@-2': [0.8876, 0.8992, 0.9077, 0.912], 'with_svm@-3': [0.8999, 0.9046, 0.9097, 0.9095], 'without_svm': [0.8667, 0.8758, 0.8939, 0.9006]}
Starting epoch: 5
Epoch 1/1
10000/10000 [==============================] - 1s 95us/step
{'with_svm@-2': [0.8876, 0.8992, 0.9077, 0.912, 0.9115], 'with_svm@-3': [0.8999, 0.9046, 0.9097, 0.9095, 0.9111],

In [ ]:
for j in range(5): 
  filename = 'performance{}.json'.format(j)
  files.download(filename)